# Part 3 : 基于LoRA（Low-Rank Adaptation）的有监督微调（Supervised Fine-tuning）
<br>

### 任务目标
1. 基于LoRA对Qwen2.5-7B模型进行微调；
2. 调整LoRA超参数，如LoRA Rank、Learning Rate，以及微调数据量。得到不同的LoRA，用于后续模型评测；
3. 通过SwanLab，对训练过程中的Loss等参数进行观测。

<br>



### LoRA原理
LLM中包含大量的线性变换层，其中参数矩阵的维度通常很高。研究发现模型在针对特定任务进行适配时，参数矩阵往往是过参数化（Over-parametrized）的，其存在一个较低的内在秩。为了解决这一问题，LoRA提出在预训练模型的参数矩阵上添加低秩分解矩阵来近似每层的参数更新，从而减少适配下游任务所需要训练的参数。给定一个参数矩阵W，其更新过程可以一般性地表达为以下形式：

$$
W = W_0 + \Delta W
$$

其中，$W_0$ 是原始参数矩阵，$\Delta W$ 是更新的梯度矩阵。LoRA 的基本思想是冻结原始矩阵 $W_0 \in \mathbb{R}^{H \times H}$，通过低秩分解矩阵 $A \in \mathbb{R}^{H \times R}$ 和 $B \in \mathbb{R}^{H \times R}$ 来近似参数更新矩阵 $\Delta W = A \cdot B^T$，其中 $R \ll H$ 是减小后的秩。

在微调期间，原始的矩阵参数 $W_0$ 不会被更新，低秩分解矩阵 $A$ 和 $B$ 则是可训练参数，用于适配下游任务。

![LoRA微调示意图](pictures/LoRA微调示意图.png)

### LoRA所需显存估算
假设LoRA需要训练的参数量为 $P_{LoRA}$，模型原始参数P。考虑到模型参数与优化器是显存占用的主要部分，这里主要考虑它们的大小，其他忽略不计。

LoRA 微调需要保存的模型参数量为2P+2$P_{LoRA}$，梯度和优化器参数总计 2$P_{LoRA}$ +4$P_{LoRA}$ +4$P_{LoRA}$ + 4$P_{LoRA}$ = 14$P_{LoRA}$，因此LoRA 微调需要的显存大小从全量微调的16P 大幅減少 2P+16$P_{LoRA}$。

一般来说，LoRA 主要被应用在每个多头注意力层的4个线性变换矩阵上，因此 $P_{LoRA}$=4•2•L•HR，其中L，H，R分别是模型层数、中间状态维度和秩。

以LLaMA（7B）（L =32，H=4096）例，常见的秩R设置为8，则 $P_{LoRA}$ =8388608，2P+16$P_{LoRA}$ = 13611048960 = 14GB， 16P =107814649856 =108GB。可以看到，模型微调需要的显存大小从 108GB 大幅下降到 14GB，能够有效减少微调模型所需要的硬件资源。考虑到 $P_{LoRA}$ $\ll$ P， 可以近似地认力轻量化微调需要的显存从 16P 降至2P。

### LoRA优势

1. 降低训练成本：与全参数微调相比，LoRA 微调在保证模型效果的同时，能够显著降低模型训练的成本；
2. 保持预训练模型的完整性：避免微调过程中覆盖或破坏原有知识，使得模型能够更好地泛化到多个任务；
3. 高效的多任务适配，降低推理成本：在多任务学习中，LoRA 允许每个任务对应一组独立的低秩矩阵，大大降低了多任务部署需求；
![多LoRA示意图](pictures/多LoRA示意图.png)


### SFT（Supervised Fine-tuning）
本质上说，SFT 所采用的词元级别训练方式是一种“行为克隆”（模仿学习的一种特殊算法）它利用教师的行为数据（即每个步骤的目标词元）作为监督标签，来直接训练大语言模型模仿教师的行为。在实现上，SFT 主要依赖于序列到序列的监督损失来优化模型。

关于 SFT，人们普遍认为其作用在于“解锁”大语言模型的能力，而非向大语言模型“注入”新能力。当待学习的标注指令数据超出了大语言模型的知识或能力范围，例如训练大语言模型回答关于模型未知事实的问题时，可能会加重模型的幻觉。

1. **优点：** 提高大语言模型在各种基准测试中的性能，增强大语言模型在不同任务上的泛化能力，提升大语言模型在专业领域的能力

2. **缺点：** 当数据超出大语言模型的知识范围时，模型易产生幻觉。通过对教师模型的蒸馏，会增加学生模型出现幻觉的可能性。不同标注者对实例数据标注的差异，会影响 SFT 的学习性能。指令数据的质量会影响大语言模型的训练效果

<br>
<br>

In [19]:
import os
import pandas as pd
import torch
from datasets import Dataset
from modelscope import snapshot_download, AutoModel, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import LoraConfig, TaskType, get_peft_model
from swanlab.integration.transformers import SwanLabCallback
import swanlab

---
<br>

## 一、环境配置

### 1.1 模型下载

使用 modelscope 中的 snapshot_download 函数下载模型，第一个参数为模型名称，参数 cache_dir 为模型的下载路径。

In [2]:
#模型下载
# model_dir = snapshot_download('Qwen/Qwen2.5-7B-Instruct', cache_dir='llm-model/')

In [20]:
model_path = "llm-model/Qwen/Qwen2___5-7B-Instruct/"
dataset_path = "dataset/sa_article_traindata_1446.json"
experiment_name = "8r_lr=1e-4_50data_epochs=4"

<br>

### 1.2 训练任务监控（SwanLab）
定义SwanLabCallback 对象，用于在模型微调过程中与 SwanLab 平台进行集成和交互，可以便捷进行实验管理、日志记录和可视化

In [21]:
swanlab_callback = SwanLabCallback(
    project="Qwen2.5-fintune",
    experiment_name=f"Qwen2.5-7B-Instruct-sa-article-{experiment_name}",
    description="使用通义千问Qwen2.5-7B-Instruct模型在sa-article-数据集上微调。",
    config={
        "model": "Qwen/Qwen2.5-7B-Instruct",
        "dataset": "dataset/",
    },
)

---
<br>

## 二、加载模型

### 2.1 加载 tokenizer 和半精度模型

本次在`V100 GPU`上，我们选择以半精度形式加载模型，如果显卡性能较好，可以用 `torch.bfolat`形式加载。对于自定义的模型一定要指定 `trust_remote_code`参数为 `True`。

In [22]:
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto",torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

<br>

### 2.2 微调前模型推理效果展示

我们按照相同的Instruction对Qwen2.5-7B进行测试，相同input问答5次，给出的结果不稳定，且不符合预期。

In [24]:
def lora_model_qwen(prompt):    
    messages = [
    {"role": "system", "content": "将以下文本按照【其他、云计算、架构师、计算机、个人娱乐、人工智能、商业案例、汽车行业、经济观察】标签纬度进行关联度评分"},
    {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to('cuda')
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(response)
    return response

In [25]:
# Demo样例数据测试原始Qwen2.5-7B模型效果
article_unproceed = "深圳福田杀出超级独角兽：估值6300亿，全球第一 关键词: 拉拉 物流 服务 平台 亿美元 2023 全球 中国 司机 公司 货运 市场 配送 商户 提供 2024 GTV 收入 中国香港 连接 核心句子: 2023年，货拉拉促成了超过5.884亿笔订单，全球货运GTV达到87.363亿美元；2024年上半年，平台促成了超过3.379亿笔订单，全球货运GTV为46.033亿美元，月活跃商户约为1520万个，月活跃司机约为140万名 历经11年发展，根据弗若斯特沙利文研究报告，货拉拉在2024年上半年实现了多个“第一”：全球闭环货运GTV（交易总额）最大、全球同城物流交易平台闭环货运GTV最大、全球平均月活跃商户最多、以及全球已完成订单数量最多的物流交易平台 3、特定行业物流平台：例如冷链物流、危险品物流、卡车物流（货车帮）、大型物流公司（安能物流）等 财务数据显示，2021年至2023年间，货拉拉的营业收入逐年增长，分别达到了8.45亿美元、10.36亿美元和13.34亿美元，而净利润则从2021年的亏损20.86亿美元逐渐转正，至2023年实现了9.73亿美元的盈利 与这三类公司相比，货拉拉有几个核心区别：1、与京东、顺丰等大型物流..."
i = 0
while i < 5:
    lora_model_qwen(article_unproceed)
    i += 1

{"标签关联度评分":商业案例}
{"标签关联度评分":商业案例}
【商业案例】：4.0【云计算】：0.0【架构师】：0.0【计算机】：0.0【个人娱乐】：0.0【人工智能】：0.0【商业案例】：4.0【汽车行业】：0.0【经济观察】：2.0 根据提供的信息，主要描述了货拉拉的发展情况及其在全球市场中的地位。因此，它与【商业案例】相关性较高，因为它详细描述了公司的财务表现和市场份额。此外，它也涉及到了一些经济观察的内容，比如公司的发展趋势和盈利状况。而与云计算、架构师、计算机、个人娱乐、人工智能和汽车行业的关系较弱。
【商业案例】3


KeyboardInterrupt: 

<br>

### 2.3 开启梯度检查
梯度检查点是一种显存优化技术，反向传播（Backpropagation）需要用到前向传播中的中间激活值（activation states）。这些中间值通常会被缓存到显存中，以供反向传播时计算梯度。然而对于大规模模型，这些中间值会占用大量显存资源。如果模型非常深或者批量较大，显存需求可能会超出硬件限制。

梯度检查点的核心思路是：**在前向传播过程中，只保留部分关键的中间激活值（称为“检查点”），释放掉其他中间值。在反向传播时，通过重新计算前向传播来获取需要的中间值。** 这是一种在显存与计算效率之间权衡的方法：显存消耗减少，因为大部分中间值没有保存。计算开销增加，因为反向传播需要额外执行部分前向计算。

如果开启梯度检查点，需要关闭use_cache，use_cache是Transformer中的一个参数，用于在推理时缓存前序计算结果（通常是注意力计算），从而加速生成

In [26]:
model.enable_input_require_grads()
model.config.use_cache = False

---
<br>

## 三、超参及训练配置

### 3.1 自定义 TrainingArguments 超参数

`TrainingArguments`要用于在使用深度学习框架（如 Hugging Face 的 Transformers）训练模型时设置训练的超参数和控制训练行为。帮助用户简化训练配置，使得代码更加模块化和易于维护。常见参数及说明如下：

- `output_dir`：模型的输出路径；
- `per_device_train_batch_size`：每个设备（如 GPU）上的训练批量大小，总的训练批量大小等于 per_device_train_batch_size × GPU 数量；
- `gradient_accumulation_steps`: 梯度累加，在多次前向传播后再执行一次反向传播和参数更新，以模拟更大的批量大小。如果显存比较小，那可以把 `batch_size` 设置小一点，梯度累加增大一些；
- `logging_steps`：多少步，输出一次 `log`；
- `num_train_epochs`：训练的轮次，指定整个数据集被训练的完整遍历次数。更大的 num_train_epochs 会让模型更充分地学习，但可能导致过拟合，需根据任务需求调整；
- `gradient_checkpointing`：梯度检查，启用后，在前向传播中释放部分中间激活值以节省显存，若启用，需确保模型支持 enable_input_require_grads 方法。

In [28]:
args = TrainingArguments(
    output_dir=f"./lora_output/Qwen2.5_instruct_lora_{experiment_name}",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=1,
    num_train_epochs=4,
    save_steps=100,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

<br>

### 3.2 定义 LoraConfig

LoRA 通过将权重矩阵分解为低秩矩阵的形式（两个小矩阵的乘积），`LoraConfig`这个类的参数可以控制这些低秩矩阵的特性，如秩（rank）。通过配置，决定对模型的哪些层（如 Transformer 的注意力层）应用 LoRA，而不是调整整个模型的参数。通过配置 LoRA 参数化策略，以实现高效、灵活的模型微调。

- `task_type`：模型类型，在这里为`TaskType.CAUSAL_LM`，代表因果语言建模任务（如 GPT 的文本生成任务）；
- `target_modules`：指定模型中哪些模块将被应用 LoRA 更新，主要就是 `attention`部分的层，不同的模型对应的层的名字不同，可以传入数组，也可以字符串，也可以正则表达式；
- `target_modules`：控制 LoRA 是否处于推理模式。推理模式。此时，LoRA 不会再训练，直接使用已训练好的低秩矩阵；
- `r`：lora的秩；
- `lora_alpha`：LoRA 的缩放因子，用于调整低秩更新矩阵的影响力。

In [29]:
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)

部分模型（如Llama系）需要将 pad_token 设置为 eos_token，即用序列结束标记（eos_token）代替填充值，是因为没有默认的 pad_token。
- `pad_token`:用于对齐序列长度（padding）。当输入序列的长度不足时，模型需要补齐到统一长度，而补齐的字符就是 pad_token；
- `eos_token`:用于表示序列的结束（end of sequence）。

In [30]:
print("Pad token:", tokenizer.pad_token)
# 若print为none，则需要设置，例如Llama类
# tokenizer.pad_token = tokenizer.eos_token

Pad token: <|endoftext|>


---
<br>

## 四、数据准备

### 4.1 数据格式化

`Lora` 训练的数据是需要经过格式化、编码之后再输入给模型进行训练。在`Pytorch` 模型训练中，一般需要将输入文本编码为 input_ids，将输出文本编码为 `labels`，编码之后的结果都是多维的向量。

定义一个预处理函数，这个函数用于对每一个样本，编码其输入、输出文本并返回一个编码后的字典，需要符合`Qwen2` 模型的 `Prompt Template`格式

```text
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
你是谁？<|im_end|>
<|im_start|>assistant
我是一个有用的助手。<|im_end|>
```

In [31]:
def process_func(example):
    MAX_LENGTH = 384    # 分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n{example['instruction']}<|im_end|>\n<|im_start|>user\n{example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

<br>

### 4.2 微调数据集编码

In [32]:
df = pd.read_json(dataset_path)[:50]  # [:50]取数据集的前多少条
ds = Dataset.from_pandas(df)
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

---
<br>

## 五、开始微调

### 5.1 封装PEFT模型
使用PEFT（Parameter-Efficient Fine-Tuning，参数高效微调）来进行LoRA微调训练。LoRA 是 PEFT 框架中的一种具体实现方法，专注于通过低秩矩阵分解的方式微调模型。在使用 LoRA 进行微调时，需要将模型包装为 PEFT 类型（例如通过 get_peft_model() 方法），可使用PEFT框架提供的统一接口和工具链。

In [33]:
model = get_peft_model(model, config)
model.print_trainable_parameters()  # 打印总训练参数

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.2643


<br>

### 5.2 使用 Trainer 训练

在 Hugging Face 的 Trainer 中，transformers框架默认会会使用线性衰减学习率（Linear Decay Scheduler）配合预热（Warmup）的学习率调度器

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)
trainer.train()

Step,Training Loss
1,14.145200


swanlab: Step 1 on key train/loss already exists, ignored.
swanlab: Step 1 on key train/grad_norm already exists, ignored.
swanlab: Step 1 on key train/learning_rate already exists, ignored.
swanlab: Step 1 on key train/epoch already exists, ignored.
swanlab: Step 2 on key train/loss already exists, ignored.
swanlab: Step 2 on key train/grad_norm already exists, ignored.
swanlab: Step 2 on key train/learning_rate already exists, ignored.
swanlab: Step 2 on key train/epoch already exists, ignored.


<br>

### 5.3 使用accelerator进行多GPU并行训练

In [16]:
# from accelerate import Accelerator
# from torch.utils.data import DataLoader
# from transformers import AdamW

# # 配置多 GPU 加速
# accelerator = Accelerator()

In [17]:
# # 数据加载器
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
# train_dataloader = DataLoader(tokenized_id, batch_size=4 * accelerator.num_processes, shuffle=True, collate_fn=data_collator)

# # 优化器
# optimizer = AdamW(model.parameters(), lr=1e-4)

# # 准备模型、数据加载器和优化器
# model, optimizer, train_dataloader = accelerator.prepare(model, optimizer, train_dataloader)

# # 训练设置
# num_epochs = 3
# gradient_accumulation_steps = 4
# logging_steps = 1
# save_steps = 100
# output_dir = f"./lora_output/Qwen2.5_instruct_lora_{experiment_name}"

# print(f"Rank: {accelerator.process_index}, Device: {accelerator.device}")

# # 训练循环
# global_step = 0
# model.train()
# for epoch in range(num_epochs):
#     for step, batch in enumerate(train_dataloader):
#         with accelerator.accumulate(model):
#             outputs = model(**{k: v.to(accelerator.device) for k, v in batch.items()})
#             loss = outputs.loss
#             accelerator.backward(loss)
#             optimizer.step()
#             optimizer.zero_grad()
        
#         # 日志记录
#         global_step += 1
#         if global_step % logging_steps == 0:
#             print(f"Epoch {epoch}, Step {global_step}, Loss: {loss.item()}")
        
#         # 保存模型
#         if global_step % save_steps == 0 and accelerator.is_main_process:
#             accelerator.save(model.state_dict(), f"{output_dir}/checkpoint-{global_step}.pth")

# # 保存最终模型
# if accelerator.is_main_process:
#     accelerator.save(model.state_dict(), f"{output_dir}/final_model.pth")